In [3]:
import pandas as pd
import requests
from datetime import datetime
from meteostat import Point, Daily

# Suppress FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Your EIA API key
EIA_API_KEY = 'NdeYqSkXdfEcdBe5v2lK3CQ3EbUmthN1plxBnGOC'

def get_renewable_production(start_date, end_date, state_code):
    base_url = 'https://api.eia.gov/v2/electricity/electric-power-operational-data/data/'
    all_data = []
    current_start = start_date
    
    while current_start <= end_date:
        params = {
            'api_key': EIA_API_KEY,
            'frequency': 'monthly',
            'data[0]': 'generation',
            'facets[fueltypeid][]': ['SUN', 'WND'],  # Solar and Wind
            'facets[location][]': [state_code],  # State code (e.g., 'NY' for New York)
            'start': current_start,
            'end': end_date,
            'sort[0][column]': 'period',
            'sort[0][direction]': 'asc',
            'length': 5000  # Maximum allowed by the API
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()['response']['data']
            all_data.extend(data)
            if len(data) < 5000:
                break
            current_start = data[-1]['period']
        else:
            print(f"Error fetching data: {response.status_code}")
            print(f"Response content: {response.text}")
            return None

    print(f"Total records fetched: {len(all_data)}")
    print(f"Date range in fetched data: from {all_data[0]['period']} to {all_data[-1]['period']}")
    
    df = pd.DataFrame(all_data)
    df['date'] = pd.to_datetime(df['period'])
    df['generation'] = pd.to_numeric(df['generation'], errors='coerce')
    
    df = df.groupby(['date', 'fueltypeid'])['generation'].sum().reset_index()
    df_pivot = df.pivot(index='date', columns='fueltypeid', values='generation')
    df_pivot.columns.name = None
    df_pivot = df_pivot.reset_index()
    return df_pivot

def get_weather_data(lat, lon, start_date, end_date):
    # Create Point for the specified location
    location = Point(lat, lon)
    
    # Get daily data
    data = Daily(location, start_date, end_date)
    data = data.fetch()
    
    # Reset index to make date a column
    data = data.reset_index()
    
    # Rename columns for consistency
    data = data.rename(columns={
        'time': 'date', # The month, represented as a date
        'tavg': 'temp', # The average air temperature in °C
        'wspd': 'wind_speed', # The average wind speed in km/h
        'prcp': 'precipitation' # The monthly precipitation total in mm
    })
    
    # Select only the columns we need
    data = data[['date', 'temp', 'wind_speed', 'precipitation']]
    
    return data

# Get data
start_date = '2021-01-01'
end_date = '2023-12-31'
state_code = 'NY'  # New York
state_coordinates = (42.6526, -73.7562)  # Albany, NY (rough center of the state)

# Fetch renewable energy production data for New York
production_data = get_renewable_production(start_date, end_date, state_code)

if production_data is not None:
    print("Renewable Energy Production Data for New York:")
    print(production_data.head())
    print(f"Date range in production data: from {production_data['date'].min()} to {production_data['date'].max()}")
    print(f"Number of months in production data: {len(production_data)}")

# Fetch weather data for New York (using Albany's coordinates)
weather_data = get_weather_data(state_coordinates[0], state_coordinates[1], 
                                datetime.strptime(start_date, '%Y-%m-%d'), 
                                datetime.strptime(end_date, '%Y-%m-%d'))
print("\nWeather Data:")
print(weather_data.head())
print(f"Date range in weather data: from {weather_data['date'].min()} to {weather_data['date'].max()}")
print(f"Number of days in weather data: {len(weather_data)}")

# Merge datasets
if production_data is not None:
    weather_data['month'] = weather_data['date'].dt.to_period('M')
    monthly_weather = weather_data.groupby('month').mean().reset_index()
    monthly_weather['date'] = monthly_weather['month'].dt.to_timestamp()

    merged_data = pd.merge(production_data, monthly_weather, on='date')

    print("\nMerged Production and Weather Data:")
    print(merged_data.head())

    # Save to CSV
    merged_data.to_csv(f'../data/{state_code}_renewable_energy_weather_data.csv', index=False)

Total records fetched: 695
Date range in fetched data: from 2021-01 to 2023-12
Renewable Energy Production Data for New York:
        date        SUN         WND
0 2021-01-01  313.85997  1309.79095
1 2021-02-01  233.49777  1589.46623
2 2021-03-01  552.00175  2874.20640
3 2021-04-01  572.01193  1642.59194
4 2021-05-01  637.82369  1384.59791
Date range in production data: from 2021-01-01 00:00:00 to 2023-12-01 00:00:00
Number of months in production data: 36

Weather Data:
        date  temp  wind_speed  precipitation
0 2021-01-02   0.4        14.5           20.1
1 2021-01-03  -1.9         5.5            3.0
2 2021-01-04   0.0         2.1            0.6
3 2021-01-05  -0.4         6.4            0.0
4 2021-01-06  -1.8        10.6            0.0
Date range in weather data: from 2021-01-02 00:00:00 to 2023-12-31 00:00:00
Number of days in weather data: 1094

Merged Production and Weather Data:
        date        SUN         WND    month       temp  wind_speed  \
0 2021-01-01  313.85997  13

In [112]:
merged_data.shape

(36, 7)